In [1]:
import os
import winsound
import numpy as np
import random
import sys
import io
import tensorflow as tf

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Input, Embedding, Dot, Reshape, LSTM


maxlen = 40
step = 3
text = []
chars = []
sentences = []
next_chars = []
x = []
y = []


In [2]:
#
# Collection of  of Functions
#

def beep():
    try:
        frequency = 250  # Set Frequency To 2500 Hertz
        duration = 100  # Set Duration To 1000 ms == 1 second
        winsound.Beep(frequency, duration)
    except Exception as e:
        pass
    else:
        pass
    finally:
        pass
    
    
def read_text_file():
    try:
        global text
        
        base_path = 'C:/Users/MB/Documents/_WimpyKids/'
        filename = 'Book1_10_tight.txt'
        path_to_file = os.path.join(base_path, filename)
        f = open(path_to_file,'rb')
        text=f.read().lower()
    except Exception as e:
        print(e) #or pass
    finally:
        print('Corpus Length: ', len(text))
        f.close()
        beep()    

        
def break_text_into_parts():
    try:
        global chars
        global x,y
        global sentences, next_chars
        
        
        chars = sorted(list(set(text)))
        print('total chars:', len(chars))
        char_indices = dict((c, i) for i, c in enumerate(chars))
        indices_char = dict((i, c) for i, c in enumerate(chars))

        # cut the text in semi-redundant sequences of maxlen characters
        maxlen = 40
        step = 3
        sentences = []
        next_chars = []
        for i in range(0, len(text) - maxlen, step):
            sentences.append(text[i: i + maxlen])
            next_chars.append(text[i + maxlen])
        print('nb sequences:', len(sentences))

        print('Vectorization...')
        x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
        y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
        for i, sentence in enumerate(sentences):
            for t, char in enumerate(sentence):
                x[i, t, char_indices[char]] = 1
            y[i, char_indices[next_chars[i]]] = 1        

    except Exception as e:
        print(e)
    finally:
        print('Length of Chars: ', len(chars))
        
        
        
def sample(preds, temperature=1.0):
    try:
        # helper function to sample an index from a probability array
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)
    except Exception as e:
        print(e) #or pass
    finally:
        pass  


def on_epoch_end(epoch, logs):
    try:
        # Function invoked at end of each epoch. Prints generated text.
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index: start_index + maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(400):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()

    except Exception as e:
        print(e) #or pass
    finally:
        pass



In [3]:
read_text_file()

Corpus Length:  1064208


In [4]:
break_text_into_parts()

total chars: 90
nb sequences: 354723
Vectorization...
Length of Chars:  90


In [5]:
# build the model: a single LSTM

print('Build model...')
print_callback = LambdaCallback(on_epoch_end = on_epoch_end)
optimizer = RMSprop(lr=0.01)

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
model.fit(x, y, batch_size = 128, epochs = 5, validation_split = 0.1, callbacks = [print_callback])
model.summary()


Build model...
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 319250 samples, validate on 35473 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/5
319232/319250 [============================>.] - ETA: 0s - loss: 1.9497 - acc: 0.4418----- Generating text after Epoch: 0
----- diversity: 0.2
must be str, not bytes
319250/319250 [==============================] - 1342s 4ms/sample - loss: 1.9496 - acc: 0.4418 - val_loss: 1.7169 - val_acc: 0.4957
Epoch 2/5
319232/319250 [============================>.] - ETA: 0s - loss: 1.6404 - acc: 0.5220----- Generating text after Epoch: 1
----- diversity: 0.2
must be str, not bytes
319250/319250 [==============================] - 1357s 4ms/sample - loss: 1.6404 - acc: 0.5220 - val_loss: 1.6302 - val_acc: 0.5172
Epoch 3/5
319232/319250 [============================>.] - ETA: 0s - loss: 1.5759 - acc: 0.5397----- Gener